In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model

In [2]:
path_train = '/content/drive/MyDrive/brain_tumor/Training'
path_test = '/content/drive/MyDrive/brain_tumor/Testing'

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalisasi nilai piksel ke rentang [0, 1]
    rotation_range=30,   # Rentang rotasi gambar
    width_shift_range=0.2,  # Pergeseran horizontal
    height_shift_range=0.2,  # Pergeseran vertikal
    shear_range=0.2,    # Pencabangan
    zoom_range=0.2,     # Zoom
    horizontal_flip=True,  # Flip horizontal
    fill_mode='nearest'  # Metode pengisian piksel saat augmentasi
)

In [4]:
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [5]:
train_generator = train_datagen.flow_from_directory(
    path_train,
    target_size=(224, 224),  # Ukuran gambar yang sesuai dengan model VGG
    batch_size=32,
    class_mode='categorical'  # Mode kelas untuk klasifikasi multi-kelas
)

Found 2870 images belonging to 4 classes.


In [6]:
test_generator = test_datagen.flow_from_directory(
    path_test,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 394 images belonging to 4 classes.


**VGG**

In [7]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 [==============================] - 1s 0us/step


In [8]:
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(4, activation='softmax')(x)

In [9]:
model = Model(inputs=base_model.input, outputs=output)

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [11]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

Epoch 1/10
89/89 [==============================] - 579s 6s/step - loss: 1.4573 - accuracy: 0.2696 - val_loss: 1.4037 - val_accuracy: 0.2917
Epoch 2/10
89/89 [==============================] - 55s 613ms/step - loss: 1.3517 - accuracy: 0.2829 - val_loss: 1.4410 - val_accuracy: 0.1901
Epoch 3/10
89/89 [==============================] - 55s 617ms/step - loss: 1.3529 - accuracy: 0.2689 - val_loss: 1.4417 - val_accuracy: 0.1901
Epoch 4/10
89/89 [==============================] - 55s 616ms/step - loss: 1.3504 - accuracy: 0.2819 - val_loss: 1.4435 - val_accuracy: 0.2474
Epoch 5/10
89/89 [==============================] - 55s 622ms/step - loss: 1.3506 - accuracy: 0.2851 - val_loss: 1.4374 - val_accuracy: 0.2552
Epoch 6/10
89/89 [==============================] - 55s 617ms/step - loss: 1.3504 - accuracy: 0.2822 - val_loss: 1.4405 - val_accuracy: 0.2552
Epoch 7/10
89/89 [==============================] - 55s 622ms/step - loss: 1.3512 - accuracy: 0.2674 - val_loss: 1.4434 - val_accuracy: 0.2552
E

In [12]:
test_loss, test_acc = model.evaluate(test_generator)
print("Akurasi pada data testing:", test_acc)

13/13 [==============================] - 3s 268ms/step - loss: 1.4453 - accuracy: 0.1878
Akurasi pada data testing: 0.1878172606229782
